# MGFD10 - Assignment 2

Date: February 9th, 2022 <br><br>
1005004503 - Youssef Iskander  <br>
1006435314 - Chin Tang Marcus Leung <br>
1005176532 - Sakif Mostafa<br>
1005149983 - Nick Ghalami<br>


In [1]:
import pandas as pd
import numpy as np

In [2]:
dfData = pd.read_csv("cross_section_us_cx_ps_v1.csv")
dfData.head(10)

,datem,dateidx,permno,exchcd,mveq,ret,mom,size,beme
0,196307,1,10006,1,134.402500,-4.516129e-02,0.698172,140.759375,1.199740
1,196307,1,10102,1,265.062375,-1.627907e-02,0.177440,269.448750,0.736371
2,196307,1,10137,1,475.070750,7.488032e-10,0.201556,475.070750,0.476324
3,196307,1,10145,1,1313.375250,2.319588e-02,0.335534,1283.601000,0.544203
4,196307,1,10161,1,516.406875,-9.404389e-02,0.354229,570.013125,0.519480
5,196307,1,10225,1,732.656250,-1.315789e-02,0.013285,742.425000,0.694987
6,196307,1,10241,1,733.033125,-2.156334e-02,0.125628,757.535625,0.751931
7,196307,1,10364,1,428.924000,5.658026e-03,0.499443,430.290000,1.152491
8,196307,1,10372,1,190.198125,2.272727e-02,0.184244,185.971500,1.931934
9,196307,1,10401,1,29141.732375,-6.217616e-03,0.222362,29324.058125,0.562632


In [3]:
dateArray = dfData["dateidx"].unique()

## Portfolios by Size

In [4]:
dfSizePortfolios = pd.DataFrame(
    columns=["Portfolio A", "Portfolio B", "Portfolio C", "Portfolio D", " Portfolio E", "Long A / Short E"])
size_long_short_count = 0

In [5]:
for i in dateArray:
    DataAtDate = dfData[dfData["dateidx"] == i]

    percentile20 = DataAtDate["size"].quantile(0.2)
    percentile40 = DataAtDate["size"].quantile(0.4)
    percentile60 = DataAtDate["size"].quantile(0.6)
    percentile80 = DataAtDate["size"].quantile(0.8)
    percentile100 = DataAtDate["size"].quantile(1)

    portfolioA = DataAtDate[DataAtDate["size"] <= percentile20]
    portfolioB = DataAtDate[(DataAtDate["size"] <= percentile40) & (DataAtDate["size"] > percentile20)]
    portfolioC = DataAtDate[(DataAtDate["size"] <= percentile60) & (DataAtDate["size"] > percentile40)]
    portfolioD = DataAtDate[(DataAtDate["size"] <= percentile80) & (DataAtDate["size"] > percentile60)]
    portfolioE = DataAtDate[(DataAtDate["size"] <= percentile100) & (DataAtDate["size"] > percentile80)]
    
    longSmall_shortBig = portfolioA["ret"].mean() - portfolioE["ret"].mean()
    if longSmall_shortBig > 0:
        size_long_short_count += 1
        
    
    
    dfSizePortfolios.loc[i] = [portfolioA["ret"].mean(), 
                               portfolioB["ret"].mean(), portfolioC["ret"].mean(), portfolioD["ret"].mean(), portfolioE["ret"].mean(), longSmall_shortBig]


In [6]:
dfSizeStats = pd.DataFrame(columns=["avg_ret", "std", "std_error", "t_stat"])

In [7]:
for col in dfSizePortfolios:
    size_mean = dfSizePortfolios[col].mean() * 100
    size_std = dfSizePortfolios[col].std() * 100
    size_stde = size_std / (dfSizePortfolios.shape[0] ** 0.5)
    size_t_stat = (size_mean - 0)/size_stde
    dfSizeStats.loc[col] = [size_mean, size_std, size_stde, size_t_stat]
dfSizeStats.head(6)

,avg_ret,std,std_error,t_stat
Portfolio A,1.377013,6.202493,0.236125,5.831717
Portfolio B,1.165816,5.349041,0.203635,5.725042
Portfolio C,1.144110,4.891836,0.186229,6.143566
Portfolio D,0.995292,4.608590,0.175446,5.672924
Portfolio E,0.911301,4.446101,0.169260,5.384027
Long A / Short E,0.465712,3.627103,0.138081,3.372734


### How many times has the long-short strategy worked with size?

In [8]:
size_fraction = size_long_short_count / dfSizePortfolios["Portfolio A"].count()
print("The Long - Short strategy is profitable with size", size_long_short_count, "out of", dfSizePortfolios["Portfolio A"].count(),
      "months. This represents ", size_fraction * 100.00, "% of the time")
print("There is no guarantee of a profit, therefore it is not an arbitrage opportunity")

The Long - Short strategy is profitable with size 383 out of 690 months. This represents  55.507246376811594 % of the time
There is no guarantee of a profit, therefore it is not an arbitrage opportunity


## Portfolios by Book-to-Market Ratio

In [9]:
dfBMPortfolios = pd.DataFrame(columns=["Portfolio A", "Portfolio B", "Portfolio C", "Portfolio D", "Portfolio E", "Long E / Short A"])
BM_long_short_count = 0

In [10]:
for i in dateArray:
    DataAtDate = dfData[dfData["dateidx"] == i]

    percentile20 = DataAtDate["beme"].quantile(0.2)
    percentile40 = DataAtDate["beme"].quantile(0.4)
    percentile60 = DataAtDate["beme"].quantile(0.6)
    percentile80 = DataAtDate["beme"].quantile(0.8)
    percentile100 = DataAtDate["beme"].quantile(1)

    portfolioA = DataAtDate[DataAtDate["beme"] <= percentile20]
    portfolioB = DataAtDate[(DataAtDate["beme"] <= percentile40) & (DataAtDate["beme"] > percentile20)]
    portfolioC = DataAtDate[(DataAtDate["beme"] <= percentile60) & (DataAtDate["beme"] > percentile40)]
    portfolioD = DataAtDate[(DataAtDate["beme"] <= percentile80) & (DataAtDate["beme"] > percentile60)]
    portfolioE = DataAtDate[(DataAtDate["beme"] <= percentile100) & (DataAtDate["beme"] > percentile80)]
    
    longE_shortA = portfolioE["ret"].mean() - portfolioA["ret"].mean()
    
    if longE_shortA > 0:
        BM_long_short_count += 1


    dfBMPortfolios.loc[i] = [portfolioA["ret"].mean(), 
                               portfolioB["ret"].mean(), portfolioC["ret"].mean(), portfolioD["ret"].mean(), portfolioE["ret"].mean(), longE_shortA]


In [11]:
dfBMStats = pd.DataFrame(columns=["avg_ret", "std", "std_error", "t_stat"])

In [12]:
for col in dfBMPortfolios:
    beme_mean = dfBMPortfolios[col].mean() * 100
    beme_std = dfBMPortfolios[col].std() * 100
    beme_stde = beme_std / (dfBMPortfolios.shape[0] ** 0.5)
    beme_t_stat = (beme_mean - 0)/beme_stde
    dfBMStats.loc[col] = [beme_mean, beme_std, beme_stde, beme_t_stat]
dfBMStats.head(6)

,avg_ret,std,std_error,t_stat
Portfolio A,0.996872,5.092429,0.193865,5.142083
Portfolio B,0.986168,4.946952,0.188327,5.236458
Portfolio C,1.137000,4.915387,0.187126,6.076130
Portfolio D,1.179609,5.094770,0.193955,6.081882
Portfolio E,1.294948,5.582710,0.212530,6.093011
Long E / Short A,0.298076,3.511135,0.133667,2.229996


### How many times has the long-short strategy worked with book-to-market ratio?

In [13]:
BM_fraction = BM_long_short_count / dfBMPortfolios["Portfolio A"].count()
print("The Long - Short strategy is profitable with size", BM_long_short_count, "out of", dfBMPortfolios["Portfolio A"].count(),
      "months. This represents ", BM_fraction * 100.00, "% of the time")
print("There is no guarantee of a profit, therefore it is not an arbitrage opportunity")

The Long - Short strategy is profitable with size 354 out of 690 months. This represents  51.30434782608696 % of the time
There is no guarantee of a profit, therefore it is not an arbitrage opportunity


## Portfolios by Momentum

In [14]:
dfMomPortfolios = pd.DataFrame(columns=["Portfolio A", "Portfolio B", "Portfolio C", "Portfolio D", "Portfolio E", "Long E / Short A"])
Mom_long_short_count = 0

In [15]:
for i in dateArray:
    DataAtDate = dfData[dfData["dateidx"] == i]

    percentile20 = DataAtDate["mom"].quantile(0.2)
    percentile40 = DataAtDate["mom"].quantile(0.4)
    percentile60 = DataAtDate["mom"].quantile(0.6)
    percentile80 = DataAtDate["mom"].quantile(0.8)
    percentile100 = DataAtDate["mom"].quantile(1)

    portfolioA = DataAtDate[DataAtDate["mom"] <= percentile20]
    portfolioB = DataAtDate[(DataAtDate["mom"] <= percentile40) & (DataAtDate["mom"] > percentile20)]
    portfolioC = DataAtDate[(DataAtDate["mom"] <= percentile60) & (DataAtDate["mom"] > percentile40)]
    portfolioD = DataAtDate[(DataAtDate["mom"] <= percentile80) & (DataAtDate["mom"] > percentile60)]
    portfolioE = DataAtDate[(DataAtDate["mom"] <= percentile100) & (DataAtDate["mom"] > percentile80)]
    
    longE_shortA = portfolioE["ret"].mean() - portfolioA["ret"].mean()
    
    if longE_shortA > 0:
        Mom_long_short_count += 1


    dfMomPortfolios.loc[i] = [portfolioA["ret"].mean(),
                             portfolioB["ret"].mean(), portfolioC["ret"].mean(), portfolioD["ret"].mean(), portfolioE["ret"].mean(), longE_shortA]


In [16]:
dfMomStats = pd.DataFrame(columns=["avg_ret", "std", "std_error", "t_stat"])

In [17]:
for col in dfBMPortfolios:
    mom_mean = dfMomPortfolios[col].mean() * 100
    mom_std = dfMomPortfolios[col].std() * 100
    mom_stde = mom_std / (dfMomPortfolios.shape[0] ** 0.5)
    mom_t_stat = (mom_mean - 0)/mom_stde
    dfMomStats.loc[col] = [mom_mean, mom_std, mom_stde, mom_t_stat]

dfMomStats.head(6)

,avg_ret,std,std_error,t_stat
Portfolio A,0.970354,6.888609,0.262245,3.700182
Portfolio B,1.078607,5.092743,0.193877,5.563347
Portfolio C,1.073674,4.627592,0.176169,6.094554
Portfolio D,1.142727,4.537339,0.172734,6.615547
Portfolio E,1.329619,5.073160,0.193132,6.884514
Long E / Short A,0.359265,5.213369,0.198470,1.810179


### How many times has the long-short strategy worked with momentum?

In [18]:
Mom_fraction = Mom_long_short_count / dfMomPortfolios["Portfolio A"].count()
print("The Long - Short strategy is profitable with size", Mom_long_short_count, "out of", dfMomPortfolios["Portfolio A"].count(),
      "months. This represents ", Mom_fraction * 100.00, "% of the time")
print("There is no guarantee of a profit, therefore it is not an arbitrage opportunity")


The Long - Short strategy is profitable with size 404 out of 690 months. This represents  58.550724637681164 % of the time
There is no guarantee of a profit, therefore it is not an arbitrage opportunity
